## Notebook to test out freesufer stats output without using freesurfer utilities

In [13]:
import pandas as pd
from freesurfer_stats import CorticalParcellationStats

In [75]:
ukbb_dkt_fields = "/home/nikhil/projects/brain_changes/brain-diff/metadata/UKBB_FS_DKT_Fields.csv"
UKBB_dkt_ct_fields = "/home/nikhil/projects/brain_changes/brain-diff/metadata/UKBB_DKT_CT_Fields.csv"

fs_output_dir = "/home/nikhil/projects/QPN_processing/test_data/fmriprep/output/freesurfer-6.0.1/"


In [89]:
stat_file = "aparc.DKTatlas.stats"
stat_measure = "average_thickness_mm" 

UKBB_dkt_ct_fields_df = pd.read_csv(UKBB_dkt_ct_fields)

hemispheres = ["lh", "rh"]

subject_id_list = ["sub-PD01369D713546","sub-NIMHANS001"]

hemi_stat_measures_dict = {}
for hemi in hemispheres:
    stat_measure_df = pd.DataFrame()
    for subject_id in subject_id_list:
        fs_stats_dir = f"{fs_output_dir}{subject_id}/stats/"
        stats = CorticalParcellationStats.read(f"{fs_stats_dir}{hemi}.{stat_file}").structural_measurements
        
        cols = ["subject_id"] + list(stats["structure_name"].values)
        vals = [subject_id] + list(stats[stat_measure].values)
        
        df = pd.DataFrame(columns=cols)
        df.loc[0] = vals
        stat_measure_df = pd.concat([stat_measure_df, df], axis=0)

    # replace columns names with ukbb field IDs
    field_df = UKBB_dkt_ct_fields_df[UKBB_dkt_ct_fields_df["hemi"]==hemi][["Field ID","roi"]]
    roi_field_id_dict = dict(zip(field_df["roi"], field_df["Field ID"]))
    stat_measure_df = stat_measure_df.rename(columns=roi_field_id_dict)
    
    hemi_stat_measures_dict[hemi] = stat_measure_df

### TODO Verify roi_field_id thickness values

In [90]:
hemi_stat_measures_dict["lh"]

,subject_id,27174,27175,27176,27177,27178,27179,27180,27181,27182,...,27195,27196,27197,27198,27199,27200,27201,27202,27203,27204
0,sub-PD01369D713546,2.487,2.599,1.931,2.784,2.842,2.402,2.719,2.319,2.245,...,2.589,2.425,2.698,2.490,2.644,2.277,2.879,2.612,2.523,2.923
0,sub-NIMHANS001,2.442,2.397,1.774,3.255,2.403,2.216,2.668,2.195,1.992,...,2.485,2.230,2.617,2.294,2.489,1.992,2.656,2.389,2.301,2.909


In [91]:
hemi_stat_measures_dict["rh"]

,subject_id,27267,27268,27269,27270,27271,27272,27273,27274,27275,...,27288,27289,27290,27291,27292,27293,27294,27295,27296,27297
0,sub-PD01369D713546,2.433,2.446,2.005,3.433,2.715,2.475,2.889,2.371,2.220,...,2.524,2.370,2.413,2.312,2.525,2.213,2.926,2.572,2.486,2.995
0,sub-NIMHANS001,2.209,2.284,1.654,3.410,2.514,2.274,2.719,2.135,2.086,...,2.496,2.088,2.628,2.326,2.541,1.958,2.693,2.422,2.178,2.884


## Read UKBB fields 
- This is based on UKBB showcase

In [71]:
ukbb_dkt_fields_df = pd.read_csv(ukbb_dkt_fields)
ukbb_dkt_fields_df.head()

,Field ID,Description
0,27143,Area of caudalanteriorcingulate (left hemisphere)
1,27236,Area of caudalanteriorcingulate (right hemisph...
2,27144,Area of caudalmiddlefrontal (left hemisphere)
3,27237,Area of caudalmiddlefrontal (right hemisphere)
4,27145,Area of cuneus (left hemisphere)


### Grab rows with specific stat measure of interest

In [72]:
stat_measure = "Mean thickness" 
ukbb_dkt_measure_df = ukbb_dkt_fields_df[ukbb_dkt_fields_df["Description"].str.startswith(stat_measure)].copy()
ukbb_dkt_measure_df["Description"] = ukbb_dkt_measure_df["Description"].str.replace("Mean thickness","Mean_thickness")
print(f"number of fields with {stat_measure}: {len(ukbb_dkt_measure_df)}")

number of fields with Mean thickness: 62


In [73]:
ukbb_dkt_measure_df[["metric","roi","hemi"]] = ukbb_dkt_measure_df["Description"].str.split(" ", expand=True)[[0,2,3]]
ukbb_dkt_measure_df["hemi"] = ukbb_dkt_measure_df["hemi"].replace({"(left":"lh", "(right":"rh"})
ukbb_dkt_measure_df.head()

,Field ID,Description,metric,roi,hemi
62,27174,Mean_thickness of caudalanteriorcingulate (lef...,Mean_thickness,caudalanteriorcingulate,lh
63,27267,Mean_thickness of caudalanteriorcingulate (rig...,Mean_thickness,caudalanteriorcingulate,rh
64,27175,Mean_thickness of caudalmiddlefrontal (left he...,Mean_thickness,caudalmiddlefrontal,lh
65,27268,Mean_thickness of caudalmiddlefrontal (right h...,Mean_thickness,caudalmiddlefrontal,rh
66,27176,Mean_thickness of cuneus (left hemisphere),Mean_thickness,cuneus,lh


In [74]:
# UKBB_dkt_ct_fields = "/home/nikhil/projects/brain_changes/brain-diff/metadata/UKBB_DKT_CT_Fields.csv"
# ukbb_dkt_measure_df.to_csv(UKBB_dkt_ct_fields)